# Trained Smart Nanogrid Model Predictor with Visualisation

In [1]:
import gym
import numpy as np
import os
import argparse
import time

import pandas as pd
import plotly
import plotly.express as px
from plotly.subplots import make_subplots, go
import plotly.io as pio
import plotly.figure_factory as ff

import json

from stable_baselines3 import PPO
from smart_nanogrid_gym.utils.config import solvers_files_directory_path

In [2]:
pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"
pd.set_option('display.max_columns', None)

In [31]:
def predict_single_day(current_model, env, kwargs):
    rewards_list = []
    actions_list = []
    

    obs, _ = env.reset(**kwargs)
    done = False
    while not done:
        action, _states = current_model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        rewards_list.append(reward)
        actions_list.append(action)

    return rewards_list, actions_list

In [3]:
environment_mode = 'prediction'
# Modes: 'no_penalty', 'on_departure', 'sparse', 'dense'
penalty_mode = 'dense'
# Variants: 
#    -'basic'(only vehicle charging from the grid)
#    -'b-pv'(vehicle charging alongside grid, solar energy from the PV system and dis/charging battery system)
model_variant = 'b-pv'

charging_mode = 'bounded'
number_of_chargers = 4

# Time intervals: '15min'(0.25), '30min'(0.5), '45min'(0.75), '1h'(1), '2h'(2)
time_interval_string = '1h'
time_interval = 1

agent_algorithm = 'PPO'
timesteps_per_episode = 24 / time_interval

prediction_episodes = 1
prediction_steps = timesteps_per_episode

device = 'cuda' if number_of_chargers >= 8 else 'cpu'

data_directory = 'single_prediction_files'

In [6]:
file_name = f'{agent_algorithm}-{model_variant}-{charging_mode}-{penalty_mode}-{number_of_chargers}ch-{time_interval_string}'
file_name

'PPO-b-pv-bounded-dense-4ch-1h'

JSON file containing INITIAL model values used

In [7]:
initial_values_path = f'solvers/RL/{data_directory}/{file_name}-initial_values.json'
initial_values_path

'solvers/RL/single_prediction_files/PPO-b-pv-bounded-dense-4ch-1h-initial_values.json'

JSON file containing PREDICTED model values

In [8]:
prediction_results_path = f'solvers/RL/{data_directory}/{file_name}-prediction_results.json'
prediction_results_path

'solvers/RL/single_prediction_files/PPO-b-pv-bounded-dense-4ch-1h-prediction_results.json'

In [17]:
env_variants = [
    {
        'variant_name': 'basic',
        'config': {
            'vehicle_to_everything': False,
            'pv_system_available_in_model': False,
            'battery_system_available_in_model': False,
            'environment_mode': 'training',
            'algorithm_used': 'PPO',
            'number_of_chargers': number_of_chargers,
            'charging_mode': charging_mode,
            'vehicle_uncharged_penalty_mode': penalty_mode,
            'time_interval': time_interval_string
        }},
    {
        'variant_name': 'b-pv',
        'config': {
            'vehicle_to_everything': False,
            'pv_system_available_in_model': True,
            'battery_system_available_in_model': True,
            'environment_mode': 'training',
            'algorithm_used': 'PPO',
            'number_of_chargers': number_of_chargers,
            'charging_mode': charging_mode,
            'vehicle_uncharged_penalty_mode': penalty_mode,
            'time_interval': time_interval_string
        }}
]

current_env = env_variants[1]
current_env_name = current_env['variant_name']

In [18]:
current_env_configuration = current_env['config']
current_env_configuration

{'vehicle_to_everything': False,
 'pv_system_available_in_model': True,
 'battery_system_available_in_model': True,
 'environment_mode': 'training',
 'algorithm_used': 'PPO',
 'number_of_chargers': 4,
 'charging_mode': 'bounded',
 'vehicle_uncharged_penalty_mode': 'dense',
 'time_interval': '1h'}

## ! Za vizualizaciju posljednjeg predviđanja za određeni tip modela - Preskočiti retke ispod do kategorije "MODEL PREDICTIONS VISUALISATION" i od tamo nastaviti pokretati kod !

In [19]:
env = gym.make('SmartNanogridEnv-v0', **current_env_configuration)

In [20]:
model_dir = f"{solvers_files_directory_path}\\RL\\models\\{file_name}"
model_path = f"{model_dir}\\999600"

In [22]:
current_env_name

'b-pv'

In [27]:
uppercase_name = file_name.upper()
new_model = None
model = None
if 'DDPG' in uppercase_name:
    new_model = DDPG.load(model_path, env=env, device=device)
    model = {'name': file_name, 'model': new_model, 'env_name': current_env_name, 'info': {'algorithm': 'DDPG'}}
elif 'PPO' in uppercase_name:
    new_model = PPO.load(model_path, env=env, device=device)
    model = {'name': file_name, 'model': new_model, 'env_name': current_env_name, 'info': {'algorithm': 'PPO'}}

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [58]:
final_rewards = {}
mean_rewards = {}
final_rewards[file_name] = 0
mean_rewards[file_name] = 0

In [59]:
reset_config = {'generate_new_initial_values': True}

In [60]:
reset_config['algorithm_used'] = model['info']['algorithm']
reset_config['environment_mode'] = 'prediction'
rewards, actions = predict_single_day(model['model'], env, reset_config)

model_name = model['name']
final_rewards[model_name] = sum(rewards)

In [61]:
mean_rewards[file_name] = np.mean(final_rewards[file_name])

### MODEL PREDICTIONS VISUALISATION

Initial values used in prediction

In [67]:
grid_tariff_high = 0.028
grid_tariff_low = 0.013333333
energy_tariff_high = 0.148933333
energy_tariff_low = 0.087613333
res_incentive = 0.014

high_tariff = grid_tariff_high + energy_tariff_high + res_incentive
low_tariff = grid_tariff_low + energy_tariff_low + res_incentive

del grid_tariff_high, energy_tariff_high, grid_tariff_low, energy_tariff_low, res_incentive

In [68]:
tariffs = []
for i in range(int(timesteps_per_episode) + 1):
    if i < 7 / time_interval or i > 19 / time_interval:
        tariffs.append(low_tariff)
    else:
        tariffs.append(high_tariff)

day_tariffs = pd.DataFrame({'Day Tariff': tariffs})

del tariffs, high_tariff, low_tariff
day_tariffs.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
Day Tariff,0.114947,0.114947,0.114947,0.114947,0.114947,0.114947,0.114947,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.190933,0.114947,0.114947,0.114947,0.114947,0.114947


In [37]:
def load_initial_values(soc, arrivals, departures, capacities, requested_soc):
    vehicles = [i for i in arrivals.columns]
    
    initial_data = pd.DataFrame()
    for charger in range(number_of_chargers):
        for vehicle in vehicles:
            arrival = arrivals.loc[arrivals.index == charger, [vehicle]]
            arrival = arrival[arrival.columns[0]].item()

            if not pd.isna(arrival):
                df = pd.DataFrame()
                df['Arrival'] = [arrival]

                departure = departures.loc[departures.index == charger, [vehicle]]
                departure = departure[departure.columns[0]].item()
                df['Departure'] = departure

                df['Vehicle ID'] = str(vehicle)
                df['Charger ID'] = str(charger)
                
                df['EV Capacity'] = capacities[charger][arrival]
                
                df['Initial State of Charge'] = soc[charger][arrival]
                df['Requested State of Charge'] = requested_soc[charger][arrival]
                
                initial_data = pd.concat([initial_data.loc[:], df]).reset_index(drop=True)

    initial_data = initial_data[['Arrival', 'Departure', 'EV Capacity', 'Initial State of Charge', 
                                 'Requested State of Charge', 'Vehicle ID', 'Charger ID']]
    return initial_data

In [62]:
with open(initial_values_path) as json_data:
    initial_data = json.load(json_data)
    
    df_charger_occupancy = pd.DataFrame(initial_data['Charger_occupancy']).T
    df_vehicle_capacities = pd.DataFrame(initial_data['Vehicle_capacities']).T
    
    initial_training_data = load_initial_values(soc=pd.DataFrame(initial_data['SOC']).T,
                                                requested_soc=pd.DataFrame(initial_data['Requested_SOC']).T,
                                                arrivals=pd.DataFrame(initial_data['Arrivals']),
                                                departures=pd.DataFrame(initial_data['Departures']),
                                                capacities=df_vehicle_capacities)
    
    del initial_data

In [63]:
charger_occupancy = pd.DataFrame()

for charger in df_charger_occupancy.columns:
    temp_df = pd.DataFrame(df_charger_occupancy[charger])
    temp_df.columns = ['Charger Occupation State']
    temp_df['Charger ID'] = charger
    charger_occupancy = pd.concat([charger_occupancy, temp_df])

del df_charger_occupancy, temp_df
# charger_occupancy

In [64]:
vehicle_capacities = pd.DataFrame()

for charger in df_vehicle_capacities.columns:
    temp_df = pd.DataFrame(df_vehicle_capacities[charger])
    temp_df.columns = ['Vehicle Capacities']
    temp_df['Charger ID'] = charger
    vehicle_capacities = pd.concat([vehicle_capacities, temp_df])

del df_vehicle_capacities, temp_df
# vehicle_capacities

Predicted values

In [69]:
with open(prediction_results_path) as json_data:
    prediction_data = json.load(json_data)
    
    df_soc = pd.DataFrame(prediction_data['SOC']).T
    df_charger_actions = pd.DataFrame(prediction_data['Charger_actions'])
    df_charger_power_values = pd.DataFrame(prediction_data['Charger_power_values'])
    
    chargers_predictions = pd.DataFrame()
    for charger in df_soc.columns:
        temp_df = pd.DataFrame(df_soc[charger])
        temp_df.columns = ['Vehicle State of Charge']
        temp_df['Charger Action'] = df_charger_actions[charger]
        temp_df['Charger Power Value'] = df_charger_power_values[charger]
        temp_df['Charger ID'] = charger
        chargers_predictions = pd.concat([chargers_predictions, temp_df])
        del temp_df
    
    del df_soc, df_charger_actions, df_charger_power_values
    
    df_battery_soc = pd.DataFrame(prediction_data['Battery_state_of_charge'])
    df_battery_action = pd.DataFrame(prediction_data['Battery_action'])
    df_battery_power_value = pd.DataFrame(prediction_data['Battery_power_value'])
    df_battery_calculated_power_value = pd.DataFrame(prediction_data['Battery_calculated_power_value'])
    
    battery_predictions= pd.DataFrame()
    battery_predictions['Battery SOC'] = df_battery_soc
    battery_predictions['Battery Action'] = df_battery_action
    battery_predictions['Battery Power Value'] = df_battery_power_value
    battery_predictions['Battery Calculated Power Value'] = df_battery_calculated_power_value
    
    del df_battery_soc, df_battery_action, df_battery_power_value, df_battery_calculated_power_value

    df_total_cost = pd.DataFrame(prediction_data['Total_cost'])
    df_total_reward = df_total_cost * (-1)
    df_total_penalties = pd.DataFrame(prediction_data['Total_penalties'])
    df_grid_energy_cost = pd.DataFrame(prediction_data['Grid_energy_cost'])
    
    costs_predictions = pd.DataFrame()
    costs_predictions['Total Cost'] = df_total_cost
    costs_predictions['Total Reward'] = df_total_reward
    costs_predictions['Total Penalty'] = df_total_penalties
    costs_predictions['Grid Energy Cost'] = df_grid_energy_cost
    costs_predictions['Electricity Day Tariffs'] = day_tariffs
    
    del df_total_cost, df_total_reward, df_grid_energy_cost, day_tariffs
    
    df_total_vehicle_penalties = pd.DataFrame(prediction_data['Total_vehicle_penalties'])
    df_insufficient_vehicle_charging_penalties = pd.DataFrame(prediction_data['Insufficiently_charged_vehicle_penalties'])
    df_needless_vehicle_charging_penalties = pd.DataFrame(prediction_data['Needlessly_charged_vehicle_penalties'])
    df_excess_vehicle_charging_penalties = pd.DataFrame(prediction_data['Excessively_charged_vehicle_penalties'])
    df_excess_vehicle_discharging_penalties = pd.DataFrame(prediction_data['Excessively_discharged_vehicle_penalties'])
    df_total_battery_penalties = pd.DataFrame(prediction_data['Total_battery_penalties'])
    df_battery_soc_below_dod_penalties = pd.DataFrame(prediction_data['Battery_SOC_below_DoD_penalties'])
    df_needless_battery_charging_penalties = pd.DataFrame(prediction_data['Needlessly_charged_battery_penalties'])
    df_needless_battery_discharging_penalties = pd.DataFrame(prediction_data['Needlessly_discharged_battery_penalties'])
    df_excess_battery_charging_penalties = pd.DataFrame(prediction_data['Excessively_charged_battery_penalties'])
    df_excess_battery_discharging_penalties = pd.DataFrame(prediction_data['Excessively_discharged_battery_penalties'])
    
    penalties = pd.DataFrame()
    penalties['Total Penalty'] = df_total_penalties
    penalties['Total Vehicle Penalty'] = df_total_vehicle_penalties
    penalties['Insufficiently Charged Vehicle Penalty'] = df_insufficient_vehicle_charging_penalties
    penalties['Needless Vehicle Charging Penalty'] = df_needless_vehicle_charging_penalties
    penalties['Excess Vehicle Charging Penalty'] = df_excess_vehicle_charging_penalties
    penalties['Excess Vehicle Discharging Penalty'] = df_excess_vehicle_discharging_penalties
    penalties['Total Battery Penalty'] = df_total_battery_penalties
    penalties['Battery SOC Below DoD Penalty'] = df_battery_soc_below_dod_penalties
    penalties['Needless Battery Charging Penalty'] = df_needless_battery_charging_penalties
    penalties['Needless Battery Discharging Penalty'] = df_needless_battery_discharging_penalties
    penalties['Excess Battery Charging Penalty'] = df_excess_battery_charging_penalties
    penalties['Excess Battery Discharging Penalty'] = df_excess_battery_discharging_penalties
    
    del df_total_penalties, df_total_vehicle_penalties, df_total_battery_penalties, df_excess_vehicle_charging_penalties
    del df_insufficient_vehicle_charging_penalties, df_excess_vehicle_discharging_penalties
    del df_needless_vehicle_charging_penalties, df_battery_soc_below_dod_penalties, df_needless_battery_charging_penalties
    del df_needless_battery_discharging_penalties, df_excess_battery_charging_penalties
    del df_excess_battery_discharging_penalties
    
    df_grid_power = pd.DataFrame(prediction_data['Grid_power'])
    df_grid_energy = pd.DataFrame(prediction_data['Grid_energy'])
    df_utilised_solar_energy = pd.DataFrame(prediction_data['Utilized_solar_energy'])
    df_available_solar_energy = pd.DataFrame(prediction_data['Available_solar_energy']).T
    df_total_charging_power = pd.DataFrame(prediction_data['Total_charging_power'])
    df_total_discharging_power = pd.DataFrame(prediction_data['Total_discharging_power'])
    
    energy_predictions = pd.DataFrame()
    energy_predictions['Grid Power'] = df_grid_power
    energy_predictions['Grid Energy'] = df_grid_energy
    energy_predictions['Utilised Solar Power'] = df_utilised_solar_energy
    energy_predictions['Available Solar Energy - 0-24h'] = df_available_solar_energy.loc[0:24]
    energy_predictions['Available Solar Energy - 24-48h'] = df_available_solar_energy.loc[24:47][0].tolist()
    energy_predictions['Total Charging Power'] = df_total_charging_power
    energy_predictions['Total Discharging Power'] = df_total_discharging_power
    
    del df_grid_power, df_grid_energy, df_utilised_solar_energy, df_available_solar_energy
    del df_total_charging_power, df_total_discharging_power
    
    del prediction_data

In [70]:
# chargers_predictions.T
# battery_predictions.T
# costs_predictions.T
# penalties.T
# energy_and_power_predictions.T

# Initial values

#### Initial Model Info

In [71]:
subplot_fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02)

max_ev_cap = vehicle_capacities['Vehicle Capacities'].max()

for v_id in initial_training_data.index:
    charger_id = int(initial_training_data.loc[v_id, 'Charger ID'])
    counter = number_of_chargers - charger_id
    charger = counter - 1
    
    arr = int(initial_training_data.loc[v_id, 'Arrival'])
    dep = int(initial_training_data.loc[v_id, 'Departure'])
    
    ev_cap_text = str(initial_training_data.loc[v_id, 'EV Capacity'])
    
    
    if initial_training_data.loc[v_id, 'Vehicle ID'] == '0':
        subplot_fig.add_scatter(line_shape='hv',
                                y=charger_occupancy.loc[charger_occupancy['Charger ID']==charger,'Charger Occupation State'], 
                                row=counter, col=1, name=f'Occupied', 
                                fillpattern=dict(shape='/', fgcolor='rgba(71, 164, 235, 0.5)', fgopacity=0.5),
                                fillcolor=f'rgba(71, 164, 235, 0.4)', 
                                line_width=0, fill='tozeroy', 
                                legendgroup='Charger occupancy', showlegend=(not v_id), 
                                legendgrouptitle=dict(text='Charger'))
        subplot_fig.add_scatter(line_shape='hv',
                                y=vehicle_capacities.loc[vehicle_capacities['Charger ID']==charger,'Vehicle Capacities']/max_ev_cap, 
                                row=counter, col=1, name='Vehicle Capacity [kWh]',
                                fillcolor=f'rgba(71, 164, 235, 0.5)', 
                                line_width=0, fill='tozeroy', legendgroup='Charger occupancy',
                                showlegend=(not v_id))
        
        subplot_fig.add_bar(x=initial_training_data.loc[(initial_training_data['Charger ID']==str(charger))&(initial_training_data.index[v_id]==v_id), 'Departure'], 
                            row=counter, col=1,
                            y=initial_training_data.loc[(initial_training_data['Charger ID']==str(charger))&(initial_training_data.index[v_id]==v_id), 'Requested State of Charge'], 
                            name=f'Requested - on departure',
                            width=0.5, marker=dict(color='teal', opacity=0.8, line_width=0), 
                            legendgroup='State of Charge', showlegend=(not v_id),
                            legendgrouptitle=dict(text='State of Charge'))
        subplot_fig.add_bar(x=initial_training_data.loc[initial_training_data['Charger ID']==str(charger), 'Arrival'], 
                            row=counter, col=1,
                            y=initial_training_data.loc[initial_training_data['Charger ID']==str(charger), 'Initial State of Charge'], 
                            name=f'Initial - on arrival', width=0.5, 
                            marker=dict(color='crimson', opacity=0.8, line_width=0), 
                            legendgroup='State of Charge', showlegend=(not v_id))
        
        subplot_fig.add_annotation(x=23, y=1.2, text=f'<i>Charger {charger_id}</i>', textangle=0, opacity=0.9, 
                                   row=charger_id+1, col=1, font_size=13, showarrow=False, font_color='grey')
    
    if dep > 24:
        subplot_fig.add_annotation(x=arr+0.5, y=0.4, text=f'<i>{ev_cap_text}</i>', textangle=-90, opacity=0.9, 
                                   font_size=14, row=charger_id+1, col=1, showarrow=False, font_color='white')
    else:
        subplot_fig.add_annotation(x=(arr+dep)/2, y=0.12, text=f'<i>{ev_cap_text}</i>', textangle=0, opacity=0.9, 
                                   font_size=14, row=charger_id+1, col=1, showarrow=False, font_color='white')
    
    subplot_fig.add_shape(type='line', x0=arr, x1=arr, y0=0, y1=1, layer='above',
                          line_color='crimson', line_width=1.5, row=charger_id+1, col=1)
    subplot_fig.add_shape(type='line', x0=dep, x1=dep, y0=0, y1=1, layer='above',
                          line_color='teal', line_width=1.5, row=charger_id+1, col=1)

subplot_fig.add_annotation(x=-0.15, y=0.5, text="State of Charge", textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)


max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Initial Model Info", barmode='overlay')
subplot_fig.update_xaxes(griddash="dot", range=[-0.2, 24.2], tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 850
plot_height = 650

subplot_fig.update_yaxes(tickformat=" ,~%", nticks=5, range=[0, 1.1], dtick=0.25)
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.43, font_size=24, y=0.9),
                          showlegend=True, legend=dict(groupclick='togglegroup', x=1.03, y=0.97))

subplot_fig.show()

del max_ev_cap, charger_id, counter, charger, arr, dep, ev_cap_text, max_timestep, plot_width, plot_height

# Predictions

#### Predicted Vehicle State of Charge

In [72]:
subplot_fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02
)

for v_id in initial_training_data.index:
    charger_id = int(initial_training_data.loc[v_id, 'Charger ID'])
    counter = number_of_chargers - charger_id
    charger = counter - 1
    
    arr = int(initial_training_data.loc[v_id, 'Arrival'])
    dep = int(initial_training_data.loc[v_id, 'Departure'])
    
    if initial_training_data.loc[v_id, 'Vehicle ID'] == '0':
        psoc = chargers_predictions.loc[chargers_predictions['Charger ID']==charger,'Vehicle State of Charge']
        subplot_fig.add_scatter(y=psoc, line_shape='linear', row=counter, col=1, name='Value at timestep end', mode='markers+lines',
                                marker=dict(size=8, symbol='circle-dot'), line=dict(color='darkturquoise'), showlegend=(not v_id), 
                                legendgrouptitle=dict(text='Predicted State of Charge'), legendgroup='Predicted Charger SOC', )
        subplot_fig.add_bar(y=psoc, row=counter, col=1, legendgroup='Predicted Charger SOC', showlegend=False, 
                            marker=dict(color=psoc, opacity=1, line_width=0, coloraxis='coloraxis'))
        
        subplot_fig.add_bar(x=initial_training_data.loc[(initial_training_data['Charger ID']==str(charger))&(initial_training_data.index[v_id]==v_id), 'Departure']-1, row=counter, col=1,
                    y=initial_training_data.loc[(initial_training_data['Charger ID']==str(charger))&(initial_training_data.index[v_id]==v_id), 'Requested State of Charge'], name=f'Requested - on departure',
                    width=0.5, marker=dict(color='teal', opacity=0.8, line_width=0), legendgroup='State of Charge', showlegend=(not v_id),
                    legendgrouptitle=dict(text='State of Charge'))
        subplot_fig.add_bar(x=initial_training_data.loc[initial_training_data['Charger ID']==str(charger), 'Arrival'], row=counter, col=1,
                    y=initial_training_data.loc[initial_training_data['Charger ID']==str(charger), 'Initial State of Charge'], name=f'Initial - on arrival',
                    width=0.5, marker=dict(color='crimson', opacity=0.8, line_width=0), legendgroup='State of Charge', showlegend=(not v_id))
        
        subplot_fig.add_annotation(x=23, y=1.2, text=f'<i>Charger {charger_id}</i>', textangle=0, opacity=0.9, row=charger_id+1, col=1,
                                   font_size=13, showarrow=False, font_color='grey')    


subplot_fig.add_annotation(x=-0.15, y=0.5, text="State of Charge", textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Predicted State of Charge per Charger",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", title='', range=[-0.4, 24.2],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 850
plot_height = 650

subplot_fig.update_yaxes(tickformat=" ,~%", nticks=5, range=[0, 1.1], dtick=0.25)
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.42, font_size=24, y=0.9), 
                          showlegend=True, 
                          coloraxis=dict(
                              colorbar=dict(len=0.7, thickness=14, yanchor='middle', y=0.35, x=1.025, 
                                            title='State of Charge', titleside='right', tickvals=[0, 0.25, 0.5, 0.75, 0.95, 0.98, 1],
                                            tickformat=',.0%', ticktext=['0%','25%','50%','75%','95%','\t  ] Optimal Value Range','100%']),
                              colorscale=[[0, 'rgba(220,20,60, 0.45)'], [0.9, 'rgba(0,128,128, 0.45)'], [0.95, 'rgba(0,206,209, 0.45)'], [1, 'rgba(0,206,209, 0.45)']]
                          )
                         )

subplot_fig.show()

del psoc, charger_id, counter, charger, arr, dep, max_timestep, plot_width, plot_height

#### Predicted Vehicle State of Charge and corresponding action and power values and penalties

In [73]:
subplot_fig = make_subplots(
    rows=5,
    row_heights=[0.17, 0.17, 0.17, 0.17, 0.32],
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.04,
    specs=[[{'secondary_y': True}], [{'secondary_y': True}], [{'secondary_y': True}], [{'secondary_y': True}],
           [{'secondary_y': True}]]
)

charger_patterns = ['', '/', '\\', '+']
charger_markers = ['circle', 'cross', 'diamond', 'square']
for v_id in initial_training_data.index:
    charger_id = int(initial_training_data.loc[v_id, 'Charger ID'])
    counter = number_of_chargers - charger_id
    charger = counter - 1
    
    arr = int(initial_training_data.loc[v_id, 'Arrival'])
    dep = int(initial_training_data.loc[v_id, 'Departure'])
    
    if initial_training_data.loc[v_id, 'Vehicle ID'] == '0':
        psoc = chargers_predictions.loc[chargers_predictions['Charger ID']==charger,'Vehicle State of Charge']
        pactions = chargers_predictions.loc[chargers_predictions['Charger ID']==charger,'Charger Action']
        ppowers = chargers_predictions.loc[chargers_predictions['Charger ID']==charger,'Charger Power Value']
        
        subplot_fig.add_bar(x=initial_training_data.loc[(initial_training_data['Charger ID']==str(charger))&(initial_training_data.index[v_id]==v_id), 'Departure']-1, row=counter, col=1,
                    y=initial_training_data.loc[(initial_training_data['Charger ID']==str(charger))&(initial_training_data.index[v_id]==v_id), 'Requested State of Charge'], name=f'Requested - on departure',
                    width=0.5, marker=dict(color='teal', opacity=0.8, line_width=0), legendgroup='State of Charge', showlegend=(not v_id),
                    legendgrouptitle=dict(text='State of Charge'), secondary_y=False)
        subplot_fig.add_bar(x=initial_training_data.loc[initial_training_data['Charger ID']==str(charger), 'Arrival'], row=counter, col=1,
                    y=initial_training_data.loc[initial_training_data['Charger ID']==str(charger), 'Initial State of Charge'], name=f'Initial - on arrival',
                    width=0.5, marker=dict(color='crimson', opacity=0.8, line_width=0), 
                            legendgroup='State of Charge', showlegend=(not v_id), secondary_y=False)
        
        subplot_fig.add_scatter(y=psoc, line_shape='linear', row=counter, col=1, name='Value at timestep end', mode='markers+lines',
                                marker=dict(size=8, symbol='circle-dot'), line=dict(color='darkturquoise'), 
                                showlegend=(not v_id), secondary_y=False,
                                legendgrouptitle=dict(text='Predicted State of Charge'), legendgroup='Predicted Charger SOC')
        subplot_fig.add_bar(y=pactions, row=counter, col=1, legendgroup='Charger Action', showlegend=True, name=f'Charger {charger}', legendgrouptitle=dict(text='Charging Action'),
                            marker=dict(color='darkturquoise', opacity=0.45, line_width=1, line_color='darkturquoise',
                                        pattern_shape=charger_patterns[charger], pattern_fgopacity=0.9, pattern_fgcolor='white', pattern_size=10), secondary_y=False)
        
        
        
        subplot_fig.add_scatter(y=ppowers, line_shape='linear', row=counter, col=1, name=f'Charger {charger}', mode='markers',
                                marker=dict(size=8, symbol=charger_markers[charger]), line=dict(color='darkorange'), 
                                showlegend=True, secondary_y=True,
                                legendgrouptitle=dict(text='Power Values'), legendgroup='Charger Power')
        
        subplot_fig.add_annotation(x=23, y=1.2, text=f'<i>Charger {charger_id}</i>', textangle=0, opacity=0.9, row=charger_id+1, col=1,
                                   font_size=13, showarrow=False, font_color='grey')    

pevpenalties = penalties['Total Vehicle Penalty']
subplot_fig.add_scatter(y=pevpenalties, line_shape='linear', row=5, col=1, name='Value at timestep end', mode='lines+markers',
                        marker=dict(size=8, symbol='circle'), line=dict(color='crimson'), fill='tozeroy',
                        showlegend=True, secondary_y=False,
                        legendgrouptitle=dict(text='Total Vehicle Penalty'), legendgroup='Total Vehicle Penalty')
subplot_fig.add_bar(x=penalties.index, y=pevpenalties, row=5, col=1, name='Value at timestep end', secondary_y=False,
                        width=0.2, legendgrouptitle=dict(text='Total Battery Penalty'), legendgroup='Total Battery Penalty',
                        marker=dict(line_width=0, opacity=0.8, color='crimson'), showlegend=False)

subplot_fig.add_annotation(x=-0.14, y=0.65, yanchor='middle', text="State of Charge - Action Value", textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=1.04, y=0.65, text="Power [kW]", textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=12/26, y=-0.08, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Predicted State of Charge with Action and Power Values",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", range=[-0.4, 24.2],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 880
plot_height = 890

total_ev_pen_max = pevpenalties.max()
y9type = 'log' if total_ev_pen_max > 100000 else 'linear'
y9max_margin = 6 if y9type == 'log' else round(total_ev_pen_max + 10)
y9yaxistext = '- log scale' if y9type == 'log' else ''
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.92), 
                          showlegend=True, legend=dict(x=26/24), margin=dict(autoexpand=True),
                          xaxis4=dict(showticklabels=True),
                          yaxis=dict(tickformat=" ,~", nticks=5, range=[0, 1.1], dtick=0.2),
                          yaxis2=dict(tickformat=" ,~", nticks=5, range=[0, 22], dtick=4),
                          yaxis3=dict(tickformat=" ,~", nticks=5, range=[0, 1.1], dtick=0.25),
                          yaxis4=dict(tickformat=" ,~", nticks=5, range=[0, 22], dtick=4),
                          yaxis5=dict(tickformat=" ,~", nticks=5, range=[0, 1.1], dtick=0.25),
                          yaxis6=dict(tickformat=" ,~", nticks=5, range=[0, 22], dtick=4),
                          yaxis7=dict(tickformat=" ,~", nticks=5, range=[0, 1.1], dtick=0.25),
                          yaxis8=dict(tickformat=" ,~", nticks=5, range=[0, 22], dtick=4),
                          yaxis9=dict(tickformat=" ,~g", nticks=9, type=y9type, range=[-1, y9max_margin], 
                                      title=dict(text=f'Penalty Value {y9yaxistext}')))

subplot_fig.show()

del psoc, charger_id, counter, charger, arr, dep, max_timestep, plot_width, plot_height

#### Vehicle Prediction Penalties per Type

In [74]:
subplot_fig = go.Figure()

pen1 = penalties['Insufficiently Charged Vehicle Penalty']
pen2 = penalties['Excess Vehicle Charging Penalty']
pen3 = penalties['Needless Vehicle Charging Penalty']

subplot_fig.add_bar(y=pen1, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='crimson', opacity=0.8, line_width=0), name='Insufficiently Charged Vehicle')
subplot_fig.add_bar(y=pen2, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='sienna', opacity=0.8, line_width=0), name='Excess Vehicle Charging')
subplot_fig.add_bar(y=pen3, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='darkorange', opacity=0.8, line_width=0), name='Needless Vehicle Charging')

pevpenalties = penalties['Total Vehicle Penalty']
subplot_fig.add_scatter(x=penalties.index, y=pevpenalties, line_shape='linear', name='Total Vehicle penalty', mode='lines+markers',
                        marker=dict(size=8, symbol='diamond', line=dict(width=1.5), color=pevpenalties, coloraxis='coloraxis'), line=dict(color='rgba(0,0,0,0.7)', width=2),
                        showlegend=True, yaxis='y', legendgroup='Total Vehicle Penalty')

subplot_fig.add_annotation(x=-0.1, y=0.5, text="Penalty Value", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Vehicle Prediction Penalties per Type",
                          yaxis=dict(title=''))
subplot_fig.update_xaxes(tickformat='', griddash="dot", title='', range=[-0.4, 24.4],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 950
plot_height = 650

subplot_fig.update_yaxes(nticks=6, dtick=0.2)
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.99), 
                          showlegend=True, coloraxis=dict(colorscale=[[0, 'crimson'], [0.5, 'orange'], [1, 'sienna']], 
                                                          colorbar=dict(title=dict(text='Total Vehicle Penalty Value', side='right'))), barmode='stack',
                          legend=dict(x=0.5, orientation='h', yanchor='bottom', entrywidth=0, y=1.01, xanchor='center'), scattermode='overlay',
                          yaxis=dict(tickformat=" ,.2~g", dtick=1, range=[-1, 6], type='log'))


subplot_fig.show()

del max_timestep, plot_width, plot_height

#### Predicted Battery System Values and Penalties

In [75]:
subplot_fig = make_subplots(
    rows=2,
    row_heights=[0.6, 0.4],
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.04,
    specs=[[{'secondary_y': True}], [{'secondary_y': True}]]
)

subplot_fig.add_scatter(y=[0.5], yaxis='y', row=1, col=1, secondary_y=False,
                        x=[0], line_shape='linear', name='Initial State of Charge %', mode='markers',
                        marker=dict(size=8, symbol='square-dot'), line=dict(color='darkorange'), showlegend=True)
subplot_fig.add_scatter(y=battery_predictions['Battery SOC'], yaxis='y', row=1, col=1, secondary_y=False,
                        x=battery_predictions.index+1, line_shape='linear', name='Current State of Charge %', mode='markers+lines',
                        marker=dict(size=8, symbol='circle-dot'), line=dict(color='darkorange'), showlegend=True)
subplot_fig.add_bar(y=battery_predictions['Battery Action'], showlegend=True, yaxis='y2', width=0.9, row=1, col=1, secondary_y=False,
                    marker=dict(color='darkturquoise', coloraxis='coloraxis', opacity=0.3, line_width=0), name='Battery Action')

subplot_fig.add_bar(y=battery_predictions['Battery Calculated Power Value'], showlegend=True, yaxis='y3', width=0.9, row=1, col=1, secondary_y=True,
                    marker=dict(color='dimgrey', pattern=dict(shape='\\', fillmode='replace', size=5), opacity=0.8, line_width=1.2, line_color='dimgrey'), name='Dis/Charging Calculated Power [kW]')
subplot_fig.add_bar(y=battery_predictions['Battery Power Value'], showlegend=True, yaxis='y3', width=0.9, row=1, col=1, secondary_y=True,
                    marker=dict(color='dimgrey', pattern=dict(shape='/', fillmode='replace', size=15), opacity=0.8, line_width=1.2, line_color='dimgrey'), name='Dis/Charging Power [kW]')

subplot_fig.add_scatter(y=battery_predictions['Battery Calculated Power Value'], line_shape='linear', secondary_y=True, 
                        name='Dis/Charging Calculated Power [kW]', mode='markers', yaxis='y3', row=1, col=1,
                        marker=dict(size=8, symbol='cross'), line=dict(color='black'), showlegend=True)
subplot_fig.add_scatter(y=battery_predictions['Battery Power Value'], line_shape='linear', secondary_y=True,
                        name='Dis/Charging Power [kW]', mode='markers', yaxis='y3', row=1, col=1,
                        marker=dict(size=8, symbol='circle'), line=dict(color='black'), showlegend=True)

pbpenalties = penalties['Total Battery Penalty']
subplot_fig.add_scatter(y=pbpenalties, line_shape='linear', row=2, col=1, name='Value at timestep end', mode='lines+markers',
                        marker=dict(size=8, symbol='circle'), line=dict(color='crimson'), fill='tozeroy',
                        showlegend=False, secondary_y=False,
                        )
subplot_fig.add_bar(x=penalties.index, y=pbpenalties, row=2, col=1, name='Value at timestep end', secondary_y=False,
                        width=0.2, legendgrouptitle=dict(text='Total Battery Penalty'), legendgroup='Total Battery Penalty',
                        marker=dict(line_width=0, opacity=0.8, color='crimson', coloraxis='coloraxis'), showlegend=True)

subplot_fig.add_annotation(x=12/26, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Predicted Battery Values",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", range=[-0.4, 24.4],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 950
plot_height = 790

total_pen_max = pbpenalties.max()
y3type = 'log' if total_pen_max > 1000000 else 'linear'
y3max_margin = 6 if y3type == 'log' else round(total_pen_max + total_pen_max/10)
y3yaxistext = '- log scale' if y3type == 'log' else ''
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=12/32, font_size=24, y=0.92), 
                          showlegend=True, legend=dict(x=26/24),
                          yaxis=dict(tickformat=" ,~", nticks=5, range=[-1.1, 1.1], dtick=0.25, 
                                     title='State of Charge - Action Value'),
                          yaxis2=dict(tickformat=" ,~", nticks=5, range=[-50.2, 50.2], dtick=10, title='Power [kW]'),
                          yaxis3=dict(tickformat=" ,~g", nticks=6, type=y3type, range=[0, y3max_margin], 
                                      title=dict(text=f'Penalty Value {y3yaxistext}')))

subplot_fig.show()

del plot_width, plot_height

#### Battery Prediction Penalties per Type

In [76]:
subplot_fig = go.Figure()

pen1 = penalties['Battery SOC Below DoD Penalty']
pen2 = penalties['Excess Battery Charging Penalty']
pen3 = penalties['Excess Battery Discharging Penalty']
pen4 = penalties['Needless Battery Charging Penalty']
pen5 = penalties['Needless Battery Discharging Penalty']

subplot_fig.add_bar(y=pen1, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='crimson', opacity=0.9, line_width=0), name='Battery SOC Below DoD')
subplot_fig.add_bar(y=pen2, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='sienna', opacity=0.9, line_width=0), name='Excess Battery Charging')
subplot_fig.add_bar(y=pen3, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='darkorange', opacity=0.9, line_width=0), name='Excess Battery Discharging')
subplot_fig.add_bar(y=pen4, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='tan', opacity=0.9, line_width=0), name='Needless Battery Charging')
subplot_fig.add_bar(y=pen5, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='darkgoldenrod', opacity=0.8, line_width=0), name='Needless Battery Discharging')

pbpenalties = penalties['Total Battery Penalty']
subplot_fig.add_scatter(x=penalties.index, y=pbpenalties, line_shape='linear', name='Total Battery penalty', mode='markers',
                        marker=dict(size=10, symbol='diamond', line=dict(width=1.5), color=pbpenalties, coloraxis='coloraxis'), line=dict(color='rgba(0,0,0,0.7)'),
                        showlegend=True, yaxis='y', legendgroup='Total Battery Penalty')

subplot_fig.add_annotation(x=-0.1, y=0.5, text="Penalty Value", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Battery Prediction Penalties per Type",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", title='', range=[-0.4, 24.4],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 980
plot_height = 650


total_pen_max = pbpenalties.max()
ytype = 'log' if total_pen_max > 100000 else 'linear'
ymax_margin = 6 if ytype == 'log' else round(total_pen_max + total_pen_max/10)
yyaxistext = '- log scale' if ytype == 'log' else ''
ydtick = 1 if ytype == 'log' else 5000
subplot_fig.update_yaxes(nticks=6, dtick=0.2)
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.99), 
                          showlegend=True, coloraxis=dict(colorscale=[[0, 'crimson'], [0.5, 'gold'], [1, 'darkgoldenrod']], 
                                                          colorbar=dict(title=dict(text='Total Battery Penalty Value', side='right'))), barmode='stack',
                          legend=dict(x=0.5, orientation='h', yanchor='bottom', entrywidth=0, y=1.01, xanchor='center'), scattermode='overlay',
                          yaxis=dict(tickformat=" ,.2~g", nticks=5, dtick=ydtick, range=[0, ymax_margin], type=f'{ytype}'))


subplot_fig.show()

del max_timestep, plot_width, plot_height

#### Total Prediction Penalties

In [77]:
subplot_fig = go.Figure()

pen1 = penalties['Total Penalty']
pen2 = penalties['Total Vehicle Penalty']
pen3 = penalties['Total Battery Penalty']

subplot_fig.add_bar(y=pen2, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='sienna', opacity=0.9, line_width=0), name='Total Vehicle Penalty')
subplot_fig.add_bar(y=pen3, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='darkorange', opacity=0.9, line_width=0), name='Total Battery Penalty')

subplot_fig.add_scatter(x=penalties.index, y=pen1, line_shape='linear', name='Total penalty', mode='lines+markers',
                        marker=dict(size=10, symbol='diamond', line=dict(width=1.5), color=pen1, coloraxis='coloraxis'), line=dict(color='rgba(0,0,0,0.7)'),
                        showlegend=True, yaxis='y', legendgroup='Total Penalty')

subplot_fig.add_annotation(x=-0.1, y=0.5, text="Penalty Value", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Prediction Penalties",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", title='', range=[-0.4, 24.4],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 980
plot_height = 650


total_pen1_max = pen1.max()
ytype = 'log' if total_pen1_max > 100000 else 'linear'
ymax_margin = 6 if ytype == 'log' else round(total_pen1_max + total_pen1_max/10)
yyaxistext = '- log scale' if ytype == 'log' else ''
ydtick = 1 if ytype == 'log' else 5000
subplot_fig.update_yaxes(nticks=6, dtick=0.2)
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.99), 
                          showlegend=True, coloraxis=dict(colorscale=[[0, 'crimson'], [0.5, 'gold'], [1, 'darkgoldenrod']], 
                                                          colorbar=dict(title=dict(text='Total Penalty Value', side='right'))), barmode='stack',
                          legend=dict(x=0.5, orientation='h', yanchor='bottom', entrywidth=0, y=1.01, xanchor='center'), scattermode='overlay',
                          yaxis=dict(tickformat=" ,.2~g", nticks=5, dtick=ydtick, range=[0, ymax_margin], type=f'{ytype}'))


subplot_fig.show()

del max_timestep, plot_width, plot_height

#### Prediction Reward - Costs and Penalties

In [78]:
subplot_fig = go.Figure()

total_cost = costs_predictions['Total Cost']
total_penalty = costs_predictions['Total Penalty']
grid_cost = costs_predictions['Grid Energy Cost']

subplot_fig.add_bar(y=total_penalty, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='crimson', opacity=0.9, line_width=0), name='Total Penalty')
subplot_fig.add_bar(y=grid_cost, showlegend=True, yaxis='y', width=0.9,
                    marker=dict(color='silver', opacity=0.9, line_width=0), name='Grid Energy Cost')

subplot_fig.add_scatter(x=costs_predictions.index, y=total_cost, line_shape='linear', name='Total Cost', mode='markers',
                        marker=dict(size=10, symbol='diamond', line=dict(width=1.5), color=total_cost, coloraxis='coloraxis'), line=dict(color='rgba(0,0,0,0.7)'),
                        showlegend=True, yaxis='y', legendgroup='Total Cost')

subplot_fig.add_annotation(x=-0.1, y=0.5, text="Cost & Penalty Value", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Prediction Costs & Penalties",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", title='', range=[-0.4, 24.4],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)])

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 980
plot_height = 650


total_cost_max = total_cost.max()
ytype = 'log' if total_cost_max > 10000 else 'linear'
ymax_margin = 6 if ytype == 'log' else round(total_cost_max + total_cost_max/10)
yyaxistext = '- log scale' if ytype == 'log' else ''
ydtick = 1 if ytype == 'log' else 5000
subplot_fig.update_yaxes(nticks=6, dtick=0.2)
subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.99), 
                          showlegend=True, coloraxis=dict(colorscale=[[0, 'rgb(22, 22, 22)'], [0.25, 'dimgrey'], [0.5, 'lightgrey'], [0.7, '#E93556'], [0.9, 'crimson'], [1, '#B81432']], 
                                                          colorbar=dict(title=dict(text='Total Cost', side='right'))), barmode='stack',
                          legend=dict(x=0.5, orientation='h', yanchor='bottom', entrywidth=0, y=1.01, xanchor='center'), scattermode='overlay',
                          yaxis=dict(tickformat=" ,.2~g", nticks=5, dtick=ydtick, range=[0, ymax_margin], type=f'{ytype}'))


subplot_fig.show()

del max_timestep, plot_width, plot_height

#### Prediction Power Values

In [79]:
subplot_fig = go.Figure()

grid_p = energy_predictions['Grid Power']
pv_p = energy_predictions['Utilised Solar Power']
total_ev_p = energy_predictions['Total Charging Power'] + energy_predictions['Total Discharging Power']
battery_p = battery_predictions['Battery Power Value']
day_tariff = costs_predictions['Electricity Day Tariffs']

subplot_fig.add_vrect(x0=0, x1=7, fillcolor='grey', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=3.5/24, y=0.05, text="Low Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='dimgrey'), showarrow=False)
subplot_fig.add_vrect(x0=7, x1=20, fillcolor='gold', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=13.5/24, y=0.05, text="Peak Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='goldenrod'), showarrow=False)
subplot_fig.add_vrect(x0=20, x1=24, fillcolor='grey', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=22/24, y=0.05, text="Low Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='dimgrey'), showarrow=False)

subplot_fig.add_scatter(x=energy_predictions.index, y=grid_p, line_shape='linear', name='Grid Power', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='turquoise', coloraxis='coloraxis'), line=dict(color='turquoise'),
                        showlegend=True, yaxis='y', legendgroup='Grid Power')
subplot_fig.add_scatter(x=energy_predictions.index, y=pv_p, line_shape='linear', name='Utilised Solar Power', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='gold', coloraxis='coloraxis'), line=dict(color='gold'),
                        showlegend=True, yaxis='y', legendgroup='Solar Power')
subplot_fig.add_scatter(x=energy_predictions.index, y=total_ev_p, line_shape='linear', name='Total Vehicle Power', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='limegreen', coloraxis='coloraxis'), line=dict(color='limegreen'),
                        showlegend=True, yaxis='y', legendgroup='Total Power')
subplot_fig.add_scatter(x=battery_predictions.index, y=battery_p, line_shape='linear', name='Total Battery Power', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='crimson', coloraxis='coloraxis'), line=dict(color='crimson'),
                        showlegend=True, yaxis='y', legendgroup='Total Power')

subplot_fig.add_scatter(x=costs_predictions.index, y=day_tariff, line_shape='hv', name='Electricity Day Tariff', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='teal', coloraxis='coloraxis'), line=dict(color='teal'),
                        showlegend=True, yaxis='y2', legendgroup='Day Tariff')

subplot_fig.add_annotation(x=-0.1, y=0.5, text="Power [kW]", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=1.07, y=0.75, text="Grid Tariff", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Prediction Power Values",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", title='', range=[-0.4, 24],
                         tickvals=[i for i in range(max_timestep)], ticktext=[str(i%24) for i in range(max_timestep)], 
                         showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2)

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 980
plot_height = 650

max_p = max(grid_p.max(), pv_p.max(), total_ev_p.max(), battery_p.max())
power_margin = int(round(max_p/10)*10)+5
power_ticks = int(power_margin/5)
max_tariff = day_tariff.max()
tariff_margin = int(max_tariff)+0.25

subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.99), 
                          showlegend=True, coloraxis=dict(colorscale=[[0, 'crimson'], [0.5, 'lightgrey'], [1, 'grey']], 
                                                          colorbar=dict(title=dict(text='', side='right'))), barmode='stack',
                          legend=dict(x=0.5, orientation='h', yanchor='bottom', entrywidth=0, y=1.01, xanchor='center'), scattermode='overlay',
                          yaxis=dict(tickformat=" ,.2~g", range=[-power_margin, power_margin], dtick=power_ticks, showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2),
                          yaxis2=dict(tickformat="$,.2f", overlaying='y', anchor='free', nticks=11, dtick=0.05, tickmode='array', tickvals=[str(i/20) for i in range(12)],
                                      autoshift=True, shift=2, side='right', range=[-tariff_margin, tariff_margin], showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2),
                          yaxis3=dict(overlaying='y',
                                      range=[0, tariff_margin], visible=False))

config = {'locales': {'en': {'format': {'currency': [" € ", ""]}}}}
subplot_fig.show(config=config)

del max_timestep, plot_width, plot_height

#### Prediction Power and separate State of Charge Values

In [80]:
subplot_fig = go.Figure()

grid_p = energy_predictions['Grid Power']
pv_p = energy_predictions['Utilised Solar Power']
total_ev_p = energy_predictions['Total Charging Power'] + energy_predictions['Total Discharging Power']
battery_soc = battery_predictions['Battery SOC']
day_tariff = costs_predictions['Electricity Day Tariffs']

subplot_fig.add_vrect(x0=0, x1=7, fillcolor='grey', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=3.5/24, y=0.05, text="Low Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='dimgrey'), showarrow=False)
subplot_fig.add_vrect(x0=7, x1=20, fillcolor='gold', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=13.5/24, y=0.05, text="Peak Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='goldenrod'), showarrow=False)
subplot_fig.add_vrect(x0=20, x1=24, fillcolor='grey', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=22/24, y=0.05, text="Low Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='dimgrey'), showarrow=False)

subplot_fig.add_scatter(x=energy_predictions.index, y=grid_p, line_shape='linear', name='Grid Power', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='turquoise', coloraxis='coloraxis'), line=dict(color='turquoise'),
                        showlegend=True, yaxis='y', legendgroup='Grid Power')
subplot_fig.add_scatter(x=energy_predictions.index, y=pv_p, line_shape='linear', name='Utilised Solar Power', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='gold', coloraxis='coloraxis'), line=dict(color='gold'),
                        showlegend=True, yaxis='y', legendgroup='Solar Power')

subplot_fig.add_scatter(x=[0, 1], y=[0.5, battery_soc[0]], line_shape='linear', name='Initial Battery State of Charge', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='limegreen', coloraxis='coloraxis'), line=dict(color='limegreen'),
                        showlegend=False, yaxis='y3', legendgroup='Battery SOC')
subplot_fig.add_scatter(x=battery_predictions.index+1, y=battery_soc, line_shape='linear', name='Battery State of Charge', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='limegreen', coloraxis='coloraxis'), line=dict(color='limegreen'),
                        showlegend=True, yaxis='y3', legendgroup='Battery SOC')

subplot_fig.add_scatter(x=costs_predictions.index, y=day_tariff, line_shape='hv', name='Electricity Day Tariff', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='teal', coloraxis='coloraxis'), line=dict(color='teal'),
                        showlegend=True, yaxis='y2', legendgroup='Day Tariff')

color = ['yellow', 'orange', 'skyblue', 'chocolate']
for ca in range(number_of_chargers):
    psoc = chargers_predictions.loc[chargers_predictions['Charger ID']==ca,'Vehicle State of Charge']
    subplot_fig.add_scatter(x=battery_predictions.index, y=psoc, line_shape='linear', name=f'Charger {ca} State of Charge', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color=ca, coloraxis='coloraxis'), line=dict(color=f'{color[ca]}'),
                        showlegend=True, yaxis='y3')

subplot_fig.add_annotation(x=-0.1, y=0.5, text="Power [kW]", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=1.065, y=0.75, text="Grid Tariff", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=1.14, y=0.75, text="State of Charge", xanchor='left', yanchor='middle', textangle=-90, xref="paper", yref="paper", font_size=14, showarrow=False)
subplot_fig.add_annotation(x=0.5, y=-0.1, text="Timestep (Hour) in a Day", textangle=0, xref="paper", yref="paper", font_size=14, showarrow=False)

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Prediction Power & State of Charge Values",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", title='', range=[-0.4, 24],
                         tickvals=[i for i in range(max_timestep)], ticktext=[str(i%24) for i in range(max_timestep)], 
                         showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2)

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 1000
plot_height = 650

max_p = max(grid_p.max(), pv_p.max(), total_ev_p.max(), battery_p.max())
power_margin = int(round(max_p/10)*10)+5
power_ticks = int(power_margin/5)
max_tariff = day_tariff.max()
tariff_margin = int(max_tariff)+0.25

subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.99), margin=dict(r=180, t=150),
                          showlegend=True, coloraxis=dict(colorscale=[[0, 'crimson'], [0.5, 'lightgrey'], [1, 'grey']],
                                                          colorbar=dict(title=dict(text='', side='right'))), barmode='stack',
                          legend=dict(x=0.5, orientation='h', yanchor='bottom', entrywidth=0, y=1.05, xanchor='center'), scattermode='overlay',
                          yaxis=dict(tickformat=" ,.2~g", range=[-power_margin*1.2, power_margin*1.2], dtick=power_ticks, showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2),
                          yaxis2=dict(tickformat="$,.2f", overlaying='y', anchor='free', nticks=11, dtick=0.05, tickmode='array', tickvals=[str(i/20) for i in range(12)],
                                      autoshift=True, shift=2, side='right', range=[-tariff_margin*1.2, tariff_margin*1.2], showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2),
                          yaxis3=dict(overlaying='y', tickformat='~%', autoshift=True, shift=25, side='right', anchor='free',
                                      range=[-1*1.2, 1*1.2], tickmode='array', tickvals=[str(i/100) for i in range(0, 120, 20)]))


config = {'locales': {'en': {'format': {'currency': [" € ", ""]}}}}
subplot_fig.show(config=config)

del max_timestep, plot_width, plot_height

#### Available Solar Energy and Utilised Solar Power

In [81]:
subplot_fig = go.Figure()

upv = energy_predictions['Utilised Solar Power']
apv = energy_predictions['Available Solar Energy - 0-24h']

subplot_fig.add_vrect(x0=0, x1=7, fillcolor='grey', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=3.5/24, y=0.05, text="Low Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='dimgrey'), showarrow=False)
subplot_fig.add_vrect(x0=7, x1=20, fillcolor='gold', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=13.5/24, y=0.05, text="Peak Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='goldenrod'), showarrow=False)
subplot_fig.add_vrect(x0=20, x1=24, fillcolor='grey', opacity=0.1, layer='above', line=dict(width=0))
subplot_fig.add_annotation(x=22/24, y=0.05, text="Low Demand", xanchor='center', yanchor='middle', xref="paper", yref="paper", font=dict(size=14, color='dimgrey'), showarrow=False)

subplot_fig.add_scatter(x=energy_predictions.index, y=upv, line_shape='linear', name='Utilised Solar Power', mode='lines+markers',
                        marker=dict(size=8, symbol='circle', line=dict(width=0.5), color='darkturquoise'), line=dict(color='darkturquoise'),
                        showlegend=True, yaxis='y', legendgroup='PV')
subplot_fig.add_scatter(x=energy_predictions.index, y=apv, line_shape='linear', name='Available Solar Energy', mode='lines+markers',
                        marker=dict(size=8, symbol='diamond', line=dict(width=0.5), color='gold'), line=dict(color='gold'),
                        showlegend=True, yaxis='y2', legendgroup='Solar')

subplot_fig.add_scatter(x=costs_predictions.index, y=day_tariff, line_shape='hv', name='Electricity Day Tariff', mode='lines+markers',
                        marker=dict(size=0, symbol='circle', line=dict(width=0), color='teal', coloraxis='coloraxis'), line=dict(color='teal'),
                        showlegend=True, yaxis='y3', legendgroup='Day Tariff')

max_timestep = int(initial_training_data.Departure.max() + 1)
subplot_fig.update_layout(title="Available Solar Energy and Utilised Solar Power",
                          yaxis=dict(title=''), barmode='overlay')
subplot_fig.update_xaxes(tickformat='', griddash="dot", title="Timestep (Hour) in a Day", range=[-0.4, 24.4],
                         tickvals=[i for i in range(max_timestep)],
                         ticktext=[str(i%24) for i in range(max_timestep)],
                         showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2)

# For plot export: w(1280)xh(720), w(1920)xh(1080)
# For better overview in jupyter: w(850)xh(650)
plot_width = 980
plot_height = 650

max_solar_power = energy_predictions['Utilised Solar Power'].max()
solar_margin = int(round(max_solar_power/2)*2)+2

max_solar_energy = energy_predictions['Available Solar Energy - 0-24h'].max()
solar_e_margin = int(round(max_solar_energy/2)*2)+2

subplot_fig.update_layout(height=plot_height, width=plot_width, title=dict(xanchor="center", x=0.5, font_size=24, y=0.99), 
                          showlegend=True, barmode='stack', margin=dict(r=180),
                          legend=dict(x=0.5, orientation='h', yanchor='bottom', entrywidth=0, y=1.01, xanchor='center'), scattermode='overlay',
                          yaxis=dict(tickformat=" ,.2~g", dtick=2, range=[0, solar_margin+5], title='Power [kW]', showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2),
                          yaxis2=dict(tickformat=" ,.2~g", overlaying='y', dtick=2, range=[0, solar_e_margin+3], 
                                      side='right', title='Energy [kWh]', showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2),
                          yaxis3=dict(tickformat="$,.2f", overlaying='y', anchor='free', dtick=0.02, title='Grid Tariff',
                                      autoshift=True, shift=10, side='right', range=[0, tariff_margin], showspikes=True, spikecolor='black', spikedash='dot', spikethickness=-2))

config = {'locales': {'en': {'format': {'currency': [" € ", ""]}}}}
subplot_fig.show(config=config)

del max_timestep, plot_width, plot_height